In [ ]:
!apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar -xvf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
%cd '/content/gdrive/MyDrive/LDS9_K265_TranHoangBach/LDS9_K265_TranHoangBach_Cuoi_ky'

Mounted at /content/gdrive
/content/gdrive/MyDrive/LDS9_K265_TranHoangBach/LDS9_K265_TranHoangBach_Cuoi_ky


In [ ]:
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import *

import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
import pandas as pd

%matplotlib inline

In [ ]:
spark = SparkSession \
            .builder \
            .master("local[*]")\
            .appName("New-Spark") \
            .config("spark.memory.fraction", 0.8) \
            .config("spark.executor.memory", "10g") \
            .config("spark.driver.memory", "10g")\
            .config("spark.sql.shuffle.partitions" , "800") \
            .config("spark.memory.offHeap.enabled",'true')\
            .config("spark.memory.offHeap.size","10g")\
            .getOrCreate()
spark

- Lựa chọn data 75000-out2.csv dưới dạng matrix.
- Đọc data bằng pandas, sau đó melt data về định dạng order_id, product_id như các bài toán Association Rules thông thường

In [ ]:
df = pd.read_csv('data/75000/75000-out2.csv', header=None)
df.columns = np.arange(-1,50)
df = df.rename(columns={-1:'order_id'})
df_melt = pd.melt(df, id_vars='order_id', var_name='product_id')
df_melt = df_melt[df_melt['value']!=0].drop(columns=['value']).sort_values('order_id')
df_melt.head()

,order_id,product_id
825000,1,11
1575000,1,21
525001,2,7
2775001,2,37
3375001,2,45


In [ ]:
schema = StructType([StructField("order_id", IntegerType(), False), 
                     StructField("product_id", IntegerType(), False),
])
orders = spark.createDataFrame(df_melt, schema=schema)
orders.show(5)

+--------+----------+
|order_id|product_id|
+--------+----------+
|       1|        11|
|       1|        21|
|       2|         7|
|       2|        37|
|       2|        45|
+--------+----------+
only showing top 5 rows



In [ ]:
orders.select('order_id').distinct().count()

75000

In [ ]:
orders.select('product_id').distinct().count()

50

- Bài toán thuộc nhóm Association Rules
- Có 75.000 giao dịch, với 50 sản phẩm.
- Để ra được kết quả các set items được mua chung thường xuyên dưới dạng đầy đủ tên Flavor and Food, ta có 2 cách để thực hiện:
    - Cách 1: Join table orders và products, sau đó thực hiện thuật toán Association trên order_id và items đầy đủ tên (Flavor & Food).
    - Cách 2: Thực hiện thuật toán Association trên order_id và items_id, sau khi ra kết quả các items_id, join với products để đổi tên các id thành tên gọi đầy đủ như yêu cầu.
- Thông thường trong thực tế sẽ chỉ thực hiện Assocation Rules trên ID, nên bài này sẽ thực hiện theo cách 2

In [ ]:
df = orders.groupby('order_id').agg(collect_set('product_id').alias('items'))
df.show(5)

+--------+--------------------+
|order_id|               items|
+--------+--------------------+
|    1580|        [12, 31, 36]|
|    1829|[15, 49, 38, 6, 7...|
|    2122|             [5, 22]|
|    3175|            [33, 42]|
|    3918|          [0, 46, 2]|
+--------+--------------------+
only showing top 5 rows



Dựa trên số lượng giao dịch, lựa chọn 1 cách tương đối
- Các items có ít nhất 2.500 giao dịch, minSupport ~ 0.03
- Các kết hợp có trên 20% xác suất xảy ra (cùng mua thêm B khi đã mua A), minConfidence = 0.2

In [ ]:
from pyspark.ml.fpm import FPGrowth

fpgrowth = FPGrowth(itemsCol='items', minSupport=0.003, minConfidence=0.2)
model = fpgrowth.fit(df)
predictions = model.transform(df)
predictions.show()

+--------+--------------------+--------------------+
|order_id|               items|          prediction|
+--------+--------------------+--------------------+
|    1580|        [12, 31, 36]|                [48]|
|    1829|[15, 49, 38, 6, 7...|[41, 24, 40, 43, ...|
|    2122|             [5, 22]|                  []|
|    3175|            [33, 42]|                  []|
|    3918|          [0, 46, 2]|                  []|
|    3997|[9, 46, 35, 24, 1...|[18, 3, 40, 41, 2...|
|    4519|        [37, 39, 47]| [17, 29, 45, 7, 11]|
|    5156|    [12, 17, 47, 29]|        [36, 31, 48]|
|    7240|     [45, 37, 7, 11]|    [15, 49, 32, 16]|
|    7340|                [23]|    [41, 24, 40, 43]|
|    7880|        [17, 29, 47]|                  []|
|    7982|          [0, 46, 2]|                  []|
|    8592|    [13, 32, 24, 40]|[41, 23, 43, 45, 16]|
|    9376|                 [1]|                [19]|
|    9465|    [12, 45, 16, 32]|[36, 31, 48, 7, 3...|
|   11458|             [38, 4]|               

Tạo bảng frequent items set

In [ ]:
freq_item = model.freqItemsets
freq_item = freq_item.withColumn('len', size(col('items')))
freq_item_together = freq_item.filter(col('len')>=2).sort(col('freq').desc())
freq_item_together.show()

+-----------+----+---+
|      items|freq|len|
+-----------+----+---+
|   [35, 18]|3982|  2|
|   [27, 28]|3819|  2|
|    [46, 0]|3303|  2|
|    [5, 22]|3294|  2|
|   [16, 32]|3263|  2|
|    [3, 18]|3253|  2|
|     [9, 4]|3236|  2|
|   [33, 42]|3230|  2|
|    [3, 35]|3227|  2|
|[3, 35, 18]|3083|  3|
|   [44, 14]|2835|  2|
|    [11, 7]|2795|  2|
|    [37, 7]|2784|  2|
|    [19, 1]|2764|  2|
|   [11, 37]|2751|  2|
|    [15, 7]|2731|  2|
|   [17, 47]|2697|  2|
|     [2, 0]|2665|  2|
|   [32, 45]|2653|  2|
|    [2, 46]|2643|  2|
+-----------+----+---+
only showing top 20 rows



Đọc data products, tạo columns Flavor-Food làm tên gọi cho từng product_id

In [ ]:
products = spark.read.csv('data/75000/goods.csv', inferSchema=True, header=True)

products = products.withColumnRenamed('Id', 'product_id')
products = products.withColumn('Flavor', regexp_replace('Flavor', "'", ""))
products = products.withColumn('Food', regexp_replace('Food', "'", ""))

products = products.withColumn('Flavor-Food', concat(col('Flavor'), lit(' '), col('Food')))
products.show(5)

+----------+----------+----+-----+------+---------------+
|product_id|    Flavor|Food|Price|  Type|    Flavor-Food|
+----------+----------+----+-----+------+---------------+
|         0| Chocolate|Cake| 8.95|'Food'| Chocolate Cake|
|         1|     Lemon|Cake| 8.95|'Food'|     Lemon Cake|
|         2|    Casino|Cake|15.95|'Food'|    Casino Cake|
|         3|     Opera|Cake|15.95|'Food'|     Opera Cake|
|         4|Strawberry|Cake|11.95|'Food'|Strawberry Cake|
+----------+----------+----+-----+------+---------------+
only showing top 5 rows



Viết function convert items thành items_label

In [ ]:
product_label = products.select('Flavor-Food').toPandas()['Flavor-Food'].tolist()
def convert_product(list_product, product_label):
    converted = []
    for product_idx in list_product:
        converted.append(product_label[product_idx])
    return converted

convert_label = udf(lambda x: convert_product(x, product_label), ArrayType(StringType()))
freq_item_together = freq_item_together.withColumn('items_label', convert_label('items'))
freq_item_together.show(truncate=False)

+-----------+----+---+-----------------------------------------+
|items      |freq|len|items_label                              |
+-----------+----+---+-----------------------------------------+
|[35, 18]   |3982|2  |[Apricot Danish, Cherry Tart]            |
|[27, 28]   |3819|2  |[Marzipan Cookie, Tuile Cookie]          |
|[46, 0]    |3303|2  |[Chocolate Coffee, Chocolate Cake]       |
|[5, 22]    |3294|2  |[Truffle Cake, Gongolais Cookie]         |
|[16, 32]   |3263|2  |[Blueberry Tart, Apricot Croissant]      |
|[3, 18]    |3253|2  |[Opera Cake, Cherry Tart]                |
|[9, 4]     |3236|2  |[Napoleon Cake, Strawberry Cake]         |
|[33, 42]   |3230|2  |[Cheese Croissant, Orange Juice]         |
|[3, 35]    |3227|2  |[Opera Cake, Apricot Danish]             |
|[3, 35, 18]|3083|3  |[Opera Cake, Apricot Danish, Cherry Tart]|
|[44, 14]   |2835|2  |[Bottled Water, Berry Tart]              |
|[11, 7]    |2795|2  |[Apple Pie, Coffee Eclair]               |
|[37, 7]    |2784|2  |[Al

- Bảng trên cho ra kết quả top các items set thường được mua chung với nhau
- Trong đó, người ta thường hay mua chung nhất các loại Cookie, Tart, Cake, Danish. 
- Điều này có thể dễ hiểu vì hầu như các loại này thường được để chung gần nhau trên kệ sản phẩm, đồng thời vì là đồ ăn nhẹ nên người ta sẽ mua nhiều loại để ăn chung cùng 1 lúc